# Generate Training Data 

In [20]:
import numpy as np
import pandas as pd
import pygeostat as gs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os

## SGS realizations 

In [24]:
Azimuth = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,170]

In [25]:
for i in Azimuth:
    imgdir = (('SGS/SGS_{}').format(i))
    gs.mkdir(imgdir)

In [3]:
sgsim = gs.Program('exes/sgsim.exe',getpar = True)

C:\Users\amokdad\New - 18\tmple9ep8st\sgsim.par has been copied to the clipboard


In [22]:
griddef = gs.GridDef(griddef="100 0 1 \n100 0 1 \n1 0.5 1")
griddef

Pygeostat GridDef:
100 0.0 1.0 
100 0.0 1.0 
1 0.5 1.0

In [9]:
Reals=0
for Azimuth in Azimuth:
    for Range in  (10,12,14,16,18,20,22,24,26,28,30):
        for Anisotropy_Ratio in  (1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0):
            for Nugget_Effect in  (0,0.1,0.2,0.3,0.4): 
                Reals = Reals+1
                sgsimpar = '''                  Parameters for SGSIM
                                                ********************

                START OF PARAMETERS:
                Uncond                               -  file with data
                2  3  0  7  0  0                     -  columns for X,Y,Z,vr,wt,sec.var.
                -1.0       1.0e21                    -  trimming limits
                0                                    -  transform the data (0=no, 1=yes)
                sgsim.trn                            -  file for output trans table
                0                                    -  consider ref. dist (0=no, 1=yes)
                histsmth.out                         -  file with ref. dist distribution
                1  2                                 -  columns for vr and wt
                0.0    15.0                          -  zmin,zmax(tail extrapolation)
                1       0.0                          -  lower tail option, parameter
                1      15.0                          -  upper tail option, parameter
                1                                    -  debugging level: 0,1,2,3
                sgsim.dbg                            -  file for debugging output
                SGS/SGS_{Azimuth}/SGSIM{Reals}.out   -  file for simulation output
                170                                  -  number of realizations to generate
                100    0.0    1.0                    -  nx,xmn,xsiz
                100    0.0    1.0                    -  ny,ymn,ysiz
                1       0.5    1.0                   -  nz,zmn,zsiz
                {Range}{Reals}                       -  random number seed
                0     48                             -  min and max original data for sim
                48                                   -  number of simulated nodes to use
                1                                    -  assign data to nodes (0=no, 1=yes)
                1     3                              -  multiple grid search (0=no, 1=yes),num
                0                                    -  maximum data per octant (0=not used)
                100.0  100.0  10.0                   -  maximum search radii (hmax,hmin,vert)
                90.0  0.0   0.0                      -  angles for search ellipsoid
                51    51    11                       -  size of covariance lookup table
                0     0.0   1.0                      -  ktype: 0=SK,1=OK,2=LVM,3=EXDR,4=COLC
                ../data/ydata.dat                    -  file with LVM, EXDR, or COLC variable
                4                                    -  column for secondary variable
                1    {Nugget_Effect}                 -  nst, nugget effect
                1    {cc}  90.0   0.0   0.0          -  it,cc,ang1,ang2,ang3
                {Range}  {a_hmin}  10                -  a_hmax, a_hmin, a_vert


                '''
                sgsim.run(parstr=sgsimpar.format(Azimuth=Azimuth,Range=Range, Anisotropy_Ratio=Anisotropy_Ratio, 
                                                 cc=1-Nugget_Effect, a_hmin=Range/Anisotropy_Ratio ,Reals=Reals))



## Sample the realizations 

In [19]:
gsample= gs.Program(program='exes/gsample',parfile='gsample.par')

In [10]:
parstr = '''       Parameters for gsample V2.000
                             *********************************
       
START OF PARAMETERS:
1                                            - Number of files to sample
SGS/SGS_{Azimuth}/SGSIM{Reals}.out           - File with first grid
1  1                                         - Number of variables and columns
{Reals}                                          - Realization to sample
100 0.0 1.0                                  - Input grid: nx,xmn,xsiz
100 0.0 1.0                                  - ny,ymn,ysiz
1   0.5  1                                   - nz,zmn,zsiz
0                                            - Sampling spacing option (0=regular, 1=random)
5 5                                          - If 0, spacing in X, Y
0.5  0.5                                     - If 0, collar of the first sample, X and Y (inside sampling grid)
80  69069                                    - If 1, number of drill holes and seed number
0  0                                         - Azimuth and dip
30                                           - Sample spacing downhole
1                                            - Starting drill hole ID for new drills
2                                            - Number of decimal places (coordinates precision, up to 6)
0.5   99                                     - Sampling grid: xmin, xmax
0.5   99                                     -                ymin, ymax
0   .5                                       -                zmin, zmax
dhs/dhs_{Azimuth}/{a}dhs{b}.out              - Output file with new drill holes
0                                            - Output keyout file? (0=no, 1=yes)
0                                            - If keyout (0=input grid inside sampling grid, 1=input blocks with assays)
keyout.out                                   - Keyout file
'''
pars = dict()
callpars = []
for i in range (1,606):
    for j in  range (1,171):
        pars['Azimuth'] = Azimuth
        pars['Reals'] = Reals
        pars['a'] = i
        pars['b'] = j
        callpars.append(dict(parstr=parstr.format(**pars)))
gs.runparallel(gsample, callpars, nprocess=3)

## Inverse distanse estimate 

In [11]:
parstr = """                Parameters for KT3DN
                                    ********************
        START OF PARAMETERS:
        dhs/dhs_{Azimuth}/{a}dhs{b}.out      -  file with data
        0  2  3  0  7  0                    -  columns for DH,X,Y,Z,var,sec var
        -998.0    1.0e21                    -  trimming limits
        0                                   -  option: 0=grid, 1=cross, 2=jackknife
        xvk.dat                             -  file with jackknife data
        1   2   0    3    0                 -  columns for X,Y,Z,vr and sec var
        kt3dn_dataspacing.out               -  data spacing analysis output file (see note)
        0    15.0                           -  number to search (0 for no dataspacing analysis, rec. 10 or 20) and composite length
        0    100   0                        -  debugging level: 0,3,5,10; max data for GSKV;output total weight of each data?(0=no,1=yes)
        kt3dn.dbg-nkt3dn.sum                -  file for debugging output (see note)
        IDW/IDW_{Azimuth}/{a}IDW{b}.out    -  file for kriged output (see GSB note)
        100   0    1.0                      -  nx,xmn,xsiz
        100   0    1.0                      -  ny,ymn,ysiz
        1    0.5    1.0                     -  nz,zmn,zsiz
        1    1      1                       -  x,y and z block discretization
        4    48    12    1                  -  min, max data for kriging,upper max for ASO,ASO incr
        0      0                            -  max per octant, max per drillhole (0-> not used)
        100.0  100.0  100.0                 -  maximum search radii
        0.0   0.0   0.0                     -  angles for search ellipsoid
        0                                   -  0=SK,1=OK,2=LVM(resid),3=LVM((1-w)*m(u))),4=colo,5=exdrift,6=ICCK
        0  0.6  0.8                     -  mean (if 0,4,5,6), corr. (if 4 or 6), var. reduction factor (if 4)
        0 0 0 0 0 0 0 0 0                   -  drift: x,y,z,xx,yy,zz,xy,xz,zy
        0                                   -  0, variable; 1, estimate trend
        extdrift.out                        -  gridded file with drift/mean
        4                                   -  column number in gridded file
        keyout.out                          -  gridded file with keyout (see note)
        0    1                              -  column (0 if no keyout) and value to keep
        -1    0.01                          -  nst, nugget effect
        1    2  0.0   0.0   0.0             -  it,cc,ang1,ang2,ang3
        20 20 20                       -  a_hmax, a_hmin, a_vert
 
        Invdist option explained:
        Set nst to -1 for inverse distance
        Inverse distance estimates are calculated with the anisotropy/angle information and weights as:
        weight(i)*=1/(h+c0(1))^(cc(1))
        The ID weights are then scaled to sum to 1
        """ 

pars = dict()
callpars = []
for i in range (1,606):
    for j in  range (1,171):
        pars['a'] = i
        pars['b'] = j
        pars['Azimuth'] = Azimuth
        callpars.append(dict(parstr=parstr.format(**pars)))
gs.runparallel(kt3dn, callpars, nprocess=5)


In [26]:
pwd

'C:\\Users\\amokdad\\New - 18'